In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
class CNN(nn.Module):
    def __init__(self, input_size=21*3*2, hidden_size=64, num_classes=24):
        super(CNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )
        
    def forward(self, x):
        return self.layers(x)

In [3]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset

class ASLAlphabet(Dataset):
    def __init__(self, root_dir, label_map=None, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        self.label_map = label_map or self._build_label_map()

        for label_name in os.listdir(root_dir):
            label_dir = os.path.join(root_dir, label_name)
            if not os.path.isdir(label_dir):
                continue
            for filename in os.listdir(label_dir):
                if filename.endswith(".npy"):
                    file_path = os.path.join(label_dir, filename)
                    label_idx = self.label_map[label_name]
                    self.samples.append((file_path, label_idx))

    def _build_label_map(self):
        labels = sorted([d for d in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir, d))])
        return {label: idx for idx, label in enumerate(labels)}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        landmarks = np.load(file_path).astype(np.float32)
        if self.transform:
            landmarks = self.transform(landmarks)
        return torch.tensor(landmarks), label


In [4]:
train_dataset=ASLAlphabet("/kaggle/input/alphabet/alphabet")
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)

In [5]:
model=CNN(input_size=21*3*2,hidden_size=64,num_classes=24)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)
num_epochs=100

In [6]:
train_losses=[]
train_accuracies = []
for epoch in range(num_epochs):
    model.train()
    running_loss=0.0
    correct=0
    total=0
    for inputs, labels in tqdm(train_loader,desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs,labels=inputs.to(device),labels.to(device)

        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    print(f"Epoch {epoch+1}/{num_epochs},", f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%,")

Epoch 1/100 [Train]: 100%|██████████| 43/43 [00:03<00:00, 10.94it/s]


Epoch 1/100, Train Loss: 3.0833, Train Acc: 8.22%,


Epoch 2/100 [Train]: 100%|██████████| 43/43 [00:00<00:00, 43.89it/s]


Epoch 2/100, Train Loss: 2.8487, Train Acc: 10.55%,


Epoch 3/100 [Train]: 100%|██████████| 43/43 [00:01<00:00, 40.54it/s]


...

Epoch 98/100, Train Loss: 0.4546, Train Acc: 91.12%,


Epoch 99/100 [Train]: 100%|██████████| 43/43 [00:00<00:00, 45.49it/s]


Epoch 99/100, Train Loss: 0.4492, Train Acc: 90.98%,


Epoch 100/100 [Train]: 100%|██████████| 43/43 [00:01<00:00, 41.79it/s]


Epoch 100/100, Train Loss: 0.4440, Train Acc: 92.14%,


In [7]:
torch.save(model.state_dict, '/kaggle/working/cnn.pth')